Imports

In [ ]:
import pandas as pd
import re
import requests
import time
import pickle

Helper Functions

In [ ]:
def get_request(CIK):
    header = {"User-agent":"nchasse@umich.edu"}
    url = 'https://data.sec.gov/api/xbrl/companyfacts/CIK' + CIK +'.json'
    r = requests.get(url,headers=header)
    data_sec = r.json()
    return data_sec

In [ ]:
def get_measures(sec):
    return sec['facts']['us-gaap'].keys()

In [ ]:
def get_measure_keys(measures,request):
    measure_keys = {}
    for measure in measures:
        measure_keys[measure] = list(request['facts']['us-gaap'][measure]['units'])[0]
    return measure_keys

In [ ]:
def get_data(measure,measure_units,request):
    quarters = []
    for dic in request['facts']['us-gaap'][measure]['units'][measure_units[measure]]:
        if ('frame' in dic.keys()):
            new_dic = {}
            new_dic['end'] = dic['end']
            new_dic['frame'] = dic['frame']
            new_dic['value'] = dic['val']
            new_dic['measure'] = measure
            if re.match(r"^CY\d{4}$",dic['frame']):
                new_dic['type'] = 'Year'
            else:
                new_dic['type'] = 'Quarter'
            quarters.append(new_dic)
    df = pd.DataFrame.from_dict(quarters)
    return df

In [ ]:
def make_dataframe(measures,measure_units,request):
    df_final = pd.DataFrame()
    for measure in measures:
        df = get_data(measure,measure_units,request)
        if len(df_final) == 0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df],join='outer')
    df_final = df_final.sort_values(by='end')
    return df_final

Apple Example

In [ ]:
apple_cik = '0000320193'
request = get_request(apple_cik)
apple_measures = get_measures(request)
apple_measure_keys = get_measure_keys(apple_measures,request)
apple_dataframe = make_dataframe(apple_measures,apple_measure_keys,request)

In [ ]:
apple_dataframe.to_pickle('apple_dataframe.pkl')

Getting Data from All Stocks Using CIK Identifier

#get all CIKS
cik = pd.read_pickle(r'C:\Users\nchas\Downloads\top500_data_nona.pkl')
cik

In [ ]:
#Run for all CIKS and then save to Pickle File